# PREREQ

In [1]:
#!pip install wandb
import wandb
import numpy as np
from tqdm.auto import tqdm
import pprint

In [2]:
wandb.init(project='test1')

wandb: Currently logged in as: ramkamal (use `wandb login --relogin` to force relogin)


In [3]:
from data_loader import load_fashion_mnist
from nn_utils import activation, neural_network, set_nn_shape

# Question 1

In [4]:
# function to log sample images onto wandb
def log_sample_images():
  dataset = load_fashion_mnist(return_images=True)
  train_images = dataset['train']['X']
  train_labels = dataset['train']['Y']

  class_names = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']
  example_images, example_labels = [], []

  label_list = list(range(10))

  for (x, y) in zip(train_images, train_labels):
    if y in label_list:
      label_list.remove(y)
      example_images.append(x)
      example_labels.append(class_names[int(y)])

  wandb.log({"Example Images:": [wandb.Image(img, caption=caption) for img, caption in zip(example_images, example_labels)]})


In [5]:
# log sample images onto wandb
log_sample_images()

# Question 2

In [6]:
class optimizer:

  @staticmethod
  def sgd_old(network, data, config):

    # num_hidden_layers, hidden_layers_size = config['num_hidden_layers'], config['hidden_layer_size']
    num_epochs, batch_size = config['num_epochs'], config['batch_size']
    eta, lambda_ = config['lr'], config['weight_decay']
    initializer, activation_func = config['weights_initializer'], config['activation']
 
    ### google the getattr function - eg: getattr(activation, 'relu')(junk) is same as activation.relu(junk)
    X_train, Y_train = data['train']['X'], data['train']['Y']
    # print(num_hidden_layers, hidden_layers_size)

    for i in range(num_epochs):
      dw = []
      db = []
      check = 0

      for k in tqdm(range(len(X_train))) :
        x = X_train[k]
        y = Y_train[k]
        temp = network.self_grad_wandb(x,y,activation_func)

        if check==0 :
          dw = temp['dw']
          db = temp['db']
          check=1

        else :
          for j in range(len(dw)) :
            dw[j] =dw[j] + temp['dw'][j]
            db[j] =db[j] + temp['db'][j]

        if (k+1)%batch_size == 0 :
          for dd in dw :
            dd *=eta
          for dd in db :
            dd *=eta
          network.update_vals(dw, db)
          check=0

  @staticmethod
  def sgd(network, data, config):

    # num_hidden_layers, hidden_layers_size = config['num_hidden_layers'], config['hidden_layer_size']
    num_epochs, batch_size = config['num_epochs'], config['batch_size']
    eta, lambda_ = config['lr'], config['weight_decay']
    initializer, activation_func = config['weights_initializer'], config['activation']
 
    ### google the getattr function - eg: getattr(activation, 'relu')(junk) is same as activation.relu(junk)
    X_train, Y_train = data['train']['X'], data['train']['Y']
    # print(num_hidden_layers, hidden_layers_size)

    for i in range(num_epochs):
      dw = []
      db = []
      check = 0

      for k in tqdm(range(0, len(X_train), batch_size)) :
        X = X_train[k: k+batch_size]
        Y = Y_train[k: k+batch_size]
        temp = network.self_grad_wandb(X, Y, activation_func)

        if check==0 :
          dw = temp['dw']
          db = temp['db']
          check=1

        else :
          for j in range(len(dw)) :
            dw[j] =dw[j] + temp['dw'][j]
            db[j] =db[j] + temp['db'][j]

        if (k+1)%batch_size == 0 :
          for dd in dw :
            dd *=eta
          for dd in db :
            dd *=eta
          network.update_vals(dw, db)
          check=0

  @staticmethod
  def momentum(network,data,config) :
    gamma = 0.9
    num_epochs, batch_size = config['num_epochs'], config['batch_size']
    eta, lambda_ = config['lr'], config['weight_decay']
    initializer, activation_func = config['weights_initializer'], config['activation']
 
    ### google the getattr function - eg: getattr(activation, 'relu')(junk) is same as activation.relu(junk)
    X_train, Y_train = data['train']['X'], data['train']['Y']
    dw = []
    db = []
    check = 0
    for i in range(max_epochs) :
      for k in tqdm(range(len(X_train))) :
        x = X_train[k]
        y = Y_train[k]
        temp = network.self_grad_wandb(x,y,activation_func)
        if check==0 :
          dw = temp['dw']
          db = temp['db']
          for dd in db :
            dd*= eta
          for dd in dw :
            dd*= eta
          check = 1
        else :
          for j in range(len(dw)) :
            dw[j] += eta*temp['dw'][j]
            db[j] += eta*temp['db'][j]
        if (k+1) % batch_size == 0 or k == len(X)-1:
          network.update_vals(dw,db)
          for dd in db :
            dd *=gamma
          for dd in dw :
            dd*=gamma

  
  '''
  def sgd1(X, Y, max_epochs,eta):
    test = set_nn_shape()
    network = neural_network(test,test)
    pred1 = []
    for i in range(max_epochs) :

      dw = []
      db = []
      for k in tqdm(range(len(X))) :
        x = X[k]
        y = Y[k]
        temp = network.self_grad_wandb(x,y)
        dw = temp['dw']
        db = temp['db']
        network.update_vals(dw,db,eta*np.ones(len(dw)),eta*np.ones(len(db)))

      pred = 0
      for x,y in zip(X,Y) :
        pred = pred + (network.predict(x)!=y)

      print('error',pred)
    
    return pred1
  '''

In [7]:
def run_callback(network,data,config) :
    
    activation_func = config['activation']
    
    X_train = data['train']['X']
    Y_train = data['train']['Y']

    X_val = data['val']['X']
    Y_val = data['val']['Y']

    X_test = data['test']['X']
    Y_test = data['test']['Y']

    train_loss = 0
    train_count = 0
    train_sq_error = 0

    for x,y in zip(X_train,Y_train) :
        temp = network.predict(x,activation_func)
        if temp['y'] == y :
            train_count += 1
        train_loss -= np.log(temp['y_hat'][y]) 
        temp['y_hat'][y] = 1 - temp['y_hat'][y]
        train_sq_error += np.sum(np.dot(temp['y_hat'],temp['y_hat']))

    val_loss = 0
    val_count = 0
    val_sq_error = 0
    for x,y in zip(X_val,Y_val) :
        temp = network.predict(x,activation_func)
        if temp['y'] == y :
            val_count += 1
        val_loss -= np.log(temp['y_hat'][y]) 
        temp['y_hat'][y] = 1 - temp['y_hat'][y]
        val_sq_error += np.sum(np.dot(temp['y_hat'],temp['y_hat'])) 
    '''
    test_loss = 0
    test_count = 0
    test_sq_error = 0
    for x,y in zip(X_test,Y_test) :
        temp = network.predict(x,activation_func)
        if temp['y'] == y :
            test_count += 1
        test_loss -= np.log(temp['y_hat'][y]) 
        temp['y_hat'][y] = 1 - temp['y_hat'][y]
        test_sq_error += np.sum(np.dot(temp['y_hat'],temp['y_hat'])) 
    '''
    return  {
        'loss': {
            'train' : np.array([train_sq_error,train_loss,train_count])/len(X_train),
            'val' : np.array([val_sq_error,val_loss,val_count])/len(X_val)
        },
        'accuracy': {
            'train': train_count / len(X_train),
            'val': val_count / len(X_val)
        }
        #'test' : np.array([test_sq_error,test_loss,test_count])/len(X_test)
    }

    



# Question 4

In [8]:
sweep_config = {
    'method': 'random',

    'parameters': {
        'num_epochs': {
            'values': [5, 10]
        },
        'num_hidden_layers': {
            'values': [3, 4, 5]
        },
        'hidden_layer_size': {
            'values': [32, 64, 128]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.5]
        },
        'lr': {
            'values': [1e-3, 1e-4, 1e-5, 1e-6]
        },
        'optimizer': {
            'values': ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam']
        },
        'batch_size': {
            'values': [16, 32, 64]
        },
        'weights_initializer': {
            'values': ['random', 'xavier']
        },
        'activation': {
            'values': ['sigmoid', 'tanh', 'relu']
        }        
    }
}

In [9]:
sweep_id = wandb.sweep(sweep_config, project='test3')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 4kd3ldac
Sweep URL: https://wandb.ai/ramkamal/test3/sweeps/4kd3ldac


In [10]:
pprint.pprint(sweep_config)

{'method': 'random',
 'parameters': {'activation': {'values': ['sigmoid', 'tanh', 'relu']},
                'batch_size': {'values': [16, 32, 64]},
                'hidden_layer_size': {'values': [32, 64, 128]},
                'lr': {'values': [0.001, 0.0001, 1e-05, 1e-06]},
                'num_epochs': {'values': [5, 10]},
                'num_hidden_layers': {'values': [3, 4, 5]},
                'optimizer': {'values': ['sgd',
                                         'momentum',
                                         'nesterov',
                                         'rmsprop',
                                         'adam',
                                         'nadam']},
                'weight_decay': {'values': [0, 0.0005, 0.5]},
                'weights_initializer': {'values': ['random', 'xavier']}}}


In [11]:
data = load_fashion_mnist()

In [12]:
class sweep_module:
  @staticmethod
  def train(config=None):

    with wandb.init(config):

      config = wandb.config
      nn_shape = set_nn_shape(False, config['num_hidden_layers'] , config['hidden_layer_size'])
      network = neural_network(nn_shape, config['weights_initializer'])
      
      # need to change this bit later to accomodate other optimization functions
      optimizer.sgd(network, data, config)
      
      # generating reports for the run
      report = run_callback(network, data, config) 
      
      # keeping track of the best model
      if (report['accuracy']['val'] > val_acc_best):
        val_acc_best, network_best = report['accuracy']['val'], network

      wandb.log({
        'batch_size': config.batch_size, 
        'val_loss' : report['loss']['val'], 
        'train_loss': report['loss']['train'],
        'train_acc': report['accuracy']['train'],
        'val_acc': report['accuracy']['val']  
      })    

In [ ]:
# for logging the best model
network_best = None
val_acc_best = -1

# performing the sweep
wandb.agent(sweep_id, sweep_module.train, count=5)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 2sc0sejz with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	hidden_layer_size: 64
wandb: 	lr: 0.0001
wandb: 	num_epochs: 10
wandb: 	num_hidden_layers: 4
wandb: 	optimizer: momentum
wandb: 	weight_decay: 0.5
wandb: 	weights_initializer: random


wandb: ERROR Run 2sc0sejz errored: ValueError('operands could not be broadcast together with shapes (10,64) (10,48) ')
wandb: Agent Starting Run: mqv6oyf9 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	hidden_layer_size: 128
wandb: 	lr: 1e-06
wandb: 	num_epochs: 5
wandb: 	num_hidden_layers: 3
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.0005
wandb: 	weights_initializer: xavier


wandb: ERROR Run mqv6oyf9 errored: ValueError('operands could not be broadcast together with shapes (10,64) (10,48) ')
wandb: Agent Starting Run: 4yiv9nuv with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	hidden_layer_size: 128
wandb: 	lr: 1e-06
wandb: 	num_epochs: 5
wandb: 	num_hidden_layers: 3
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.0005
wandb: 	weights_initializer: random


wandb: ERROR Run 4yiv9nuv errored: ValueError('operands could not be broadcast together with shapes (10,64) (10,48) ')
wandb: Agent Starting Run: 9p7j3vaz with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	hidden_layer_size: 64
wandb: 	lr: 0.001
wandb: 	num_epochs: 5
wandb: 	num_hidden_layers: 4
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.5
wandb: 	weights_initializer: random


wandb: ERROR Run 9p7j3vaz errored: ValueError('operands could not be broadcast together with shapes (10,64) (10,48) ')
wandb: Agent Starting Run: bieg5x2d with config:
wandb: 	activation: relu
wandb: 	batch_size: 16
wandb: 	hidden_layer_size: 128
wandb: 	lr: 1e-06
wandb: 	num_epochs: 10
wandb: 	num_hidden_layers: 5
wandb: 	optimizer: momentum
wandb: 	weight_decay: 0.0005
wandb: 	weights_initializer: xavier


# **WORK PENDING**

0. vectorize the propogation algorithms
1. 4 remaining optimizers
2. regularization
3. check if the required graphs are retrievable
4. run the full model
5. do confushon matrix
6. interpretting the confusion matrix
7. repeat process for squared error
8. repeat process for mnist dataset

1. Weights
2. Biases
3. a
4. h
5. Initialize weights and biases
6. Optimizer

In [ ]:
import tensorflow as tf

In [ ]:
initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)

In [ ]:
for i in range(5):
    values = initializer(shape=(2, 2)).numpy()
    print(values)

In [ ]:
Y = np.array([[1, 2, 4]]).T

In [ ]:
A = np.zeros((10, 3))

In [ ]:
A

In [ ]:
A[Y.T, np.arange(3)] = 1

In [ ]:
A

In [ ]:
np.arange(5)

In [ ]:
Y

In [ ]:
E = np.zeros((10, 3))

In [ ]:
E

In [ ]:
for i in range(len(Y)):
    E[int(Y[i])][i] = 1

In [ ]:
E